<a href="https://colab.research.google.com/github/limshaocong/SysBERT/blob/main/Sequence_classification_PROMISE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 7.0 MB/s 
     |████████████████████████████████| 4.2 MB 68.7 MB/s 
     |████████████████████████████████| 140 kB 56.6 MB/s 
     |████████████████████████████████| 1.1 MB 54.8 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 62.6 MB/s 
     |████████████████████████████████| 596 kB 65.2 MB/s 
     |████████████████████████████████| 127 kB 56.7 MB/s 
     |████████████████████████████████| 6.6 MB 53.4 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 70.0 MB/s 
     |████████████████████████████████| 144 kB 54.2 MB/s 
     |████████████████████████████████| 112 kB 82.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling

In [2]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-cd9a3d34-15ab-9a2c-1cce-ef3e194cc1d0)


# Import & Pre-process Data

Clone the repository containing the labeled requirements data.

In [3]:
! git clone https://github.com/limshaocong/SysBERT/

Cloning into 'SysBERT'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (526/526), done.
remote: Compressing objects: 100% (492/492), done.
remote: Total 526 (delta 71), reused 434 (delta 22), pack-reused 0
Receiving objects: 100% (526/526), 6.68 MiB | 8.81 MiB/s, done.
Resolving deltas: 100% (71/71), done.


This sequence classification task is performed using the labeled PROMISE dataset. The targte variable is denoted by the 'is_functional' column; 1 = functional requirement, 0 = non-functional requirement. The train, validation and test datasets are created by stratified sampling in a 70/15/15 ratio. The data is imported as a HuggingFace [Dataset](https://huggingface.co/docs/datasets/access) object for ease of downstream manipulation.

In [4]:
from datasets import load_dataset

train_path = '/content/SysBERT/Requirements/Labeled/Sequence_Classification/Pre-processed/Full/train.csv'
val_path = '/content/SysBERT/Requirements/Labeled/Sequence_Classification/Pre-processed/Full/val.csv'
test_path = '/content/SysBERT/Requirements/Labeled/Sequence_Classification/Pre-processed/Full/test.csv'

ds = load_dataset(
    'csv',
    data_files = {
        'train': train_path,
        'val' : val_path,
        'test': test_path
        }
)

ds

Using custom data configuration default-034be2e3c24945ac


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-034be2e3c24945ac/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 669
    })
    val: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 143
    })
    test: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 144
    })
})

To transform natural language requirements into a BERT-compatible format, the text must first be tokenized. This is performed using a pre-trained tokenizer.

In [26]:
from transformers import AutoTokenizer

# model_checkpoint = 'bert-base-cased'
model_checkpoint = 'roberta-base'

tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    use_fast = True
)

def encode(requirements):
    return tokenizer(requirements['reqs'], truncation = True, max_length = 128)

tokenized_ds = ds.map(encode, batched = True)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [27]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'attention_mask'],
        num_rows: 669
    })
    val: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'attention_mask'],
        num_rows: 143
    })
    test: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'attention_mask'],
        num_rows: 144
    })
})

In [28]:
sample = ds['train'][0]

print(sample)
encode(sample)

{'reqs': 'The RFS system should be available 24/7 especially during the budgeting period. The RFS system shall be available 90% of the time all year and 98% during the budgeting period. 2% of the time the system will become available within 1 hour of the time that the situation is reported.', 'is_functional': 0}


{'input_ids': [0, 133, 248, 7881, 467, 197, 28, 577, 706, 73, 406, 941, 148, 5, 1229, 154, 675, 4, 20, 248, 7881, 467, 5658, 28, 577, 1814, 207, 9, 5, 86, 70, 76, 8, 8757, 207, 148, 5, 1229, 154, 675, 4, 132, 207, 9, 5, 86, 5, 467, 40, 555, 577, 624, 112, 1946, 9, 5, 86, 14, 5, 1068, 16, 431, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
from transformers import DataCollatorWithPadding

batch_size = 16
output_col = 'is_functional'

data_collator = DataCollatorWithPadding(
    tokenizer = tokenizer,
    return_tensors = 'tf'
)

batched_train_ds = tokenized_ds['train'].to_tf_dataset(
    columns = ['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols = [output_col],
    shuffle = False,
    drop_remainder = False,
    collate_fn = data_collator,
    batch_size = batch_size
)

batched_val_ds = tokenized_ds['val'].to_tf_dataset(
    columns = ['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols = [output_col],
    shuffle = False,
    drop_remainder = False,
    collate_fn = data_collator,
    batch_size = batch_size
)

batched_test_ds = tokenized_ds['test'].to_tf_dataset(
    columns = ['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols = [output_col],
    shuffle = False,
    drop_remainder = False,
    collate_fn = data_collator,
    batch_size = batch_size
)

# Model Fine-tuning

In [30]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, create_optimizer

tf.keras.utils.set_random_seed(1)
num_epochs = 10

def create_model():

  model = TFAutoModelForSequenceClassification.from_pretrained(
      model_checkpoint,
      num_labels = 2
  )

  batches_per_epoch = len(tokenized_ds['train']) // batch_size
  total_train_steps = int(batches_per_epoch * num_epochs)

  optimizer, schedule = create_optimizer(
      init_lr = 2e-5,
      num_warmup_steps = 1000,
      num_train_steps = total_train_steps,
      weight_decay_rate = 0.01
  )

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

  model.compile(
      optimizer = optimizer,
      loss = loss,
      metrics = tf.metrics.SparseCategoricalAccuracy()
  )

  return model

model = create_model()

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
from transformers import models

for layer in model.layers[:]:
    print(layer, layer.trainable)

print('=========================================================================')

frozen_layers = range(0, 4)

for layer in model.layers[:]:
  
  # Replace transformers.models.bert.modeling_tf_bert.TFBertMainLayer
  # with the corresponding MainLayer name from the previous code output
  if isinstance(layer, models.bert.modeling_tf_bert.TFBertMainLayer):
    
    for idx, layer in enumerate(layer.encoder.layer):
      
      if idx in frozen_layers:
        layer.trainable = False
      
      # Confirm the chosen layers are frozen
      print(layer, layer.trainable)

<transformers.models.roberta.modeling_tf_roberta.TFRobertaMainLayer object at 0x7f7ee00c66d0> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaClassificationHead object at 0x7f7e11af5b90> True


In [32]:
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________


In [33]:
import math
import os
import numpy as np
from tensorflow.keras.callbacks import Callback, CSVLogger, ModelCheckpoint
from sklearn.metrics import f1_score

class macro_F1(Callback):

    def __init__(self):    
        super(macro_F1, self).__init__()

    def on_epoch_end(self, epoch, logs = {}):

        y_train_true = tokenized_ds['train']['is_functional']
        y_train_pred = np.argmax(self.model.predict(batched_train_ds)['logits'], axis = 1)
        logs['train_macro_f1'] = f1_score(y_train_true, y_train_pred)

        y_val_true = tokenized_ds['val']['is_functional']
        y_val_pred = np.argmax(self.model.predict(batched_val_ds)['logits'], axis = 1)
        logs['val_macro_f1'] = f1_score(y_val_true, y_val_pred)

macro_F1_cb = macro_F1()

csvlogger_file = f'reqbert-frnfr-results.csv'
csvlogger_cb = CSVLogger(csvlogger_file)

checkpoint_path = 'model_checkpoints/reqbert-epoch{epoch}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

modelcheckpoint_cb = ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only = True,
    verbose = 1
)

In [34]:
callbacks = [macro_F1_cb, csvlogger_cb, modelcheckpoint_cb]

In [35]:
model.fit(
    batched_train_ds,
    validation_data = batched_val_ds,
    epochs = num_epochs,
    callbacks = callbacks
)

Epoch 1/10
42/42 [==============================] - ETA: 0s - loss: 0.6805 - sparse_categorical_accuracy: 0.5889
Epoch 1: saving model to model_checkpoints/reqbert-epoch1.ckpt
42/42 [==============================] - 39s 509ms/step - loss: 0.6805 - sparse_categorical_accuracy: 0.5889 - val_loss: 0.6783 - val_sparse_categorical_accuracy: 0.6154 - train_macro_f1: 0.7565 - val_macro_f1: 0.7619
Epoch 2/10
42/42 [==============================] - ETA: 0s - loss: 0.6777 - sparse_categorical_accuracy: 0.6084
Epoch 2: saving model to model_checkpoints/reqbert-epoch2.ckpt
42/42 [==============================] - 15s 368ms/step - loss: 0.6777 - sparse_categorical_accuracy: 0.6084 - val_loss: 0.6730 - val_sparse_categorical_accuracy: 0.6154 - train_macro_f1: 0.7565 - val_macro_f1: 0.7619
Epoch 3/10
42/42 [==============================] - ETA: 0s - loss: 0.6720 - sparse_categorical_accuracy: 0.6084
Epoch 3: saving model to model_checkpoints/reqbert-epoch3.ckpt
42/42 [=============================

In [24]:
# y_true = tokenized_ds['test']['is_functional']
# y_pred = np.argmax(model.predict(batched_test_ds)['logits'], axis = 1)
# micro_f1 = f1_score(y_true, y_pred)

# print(f'Test macro F1: {micro_f1:2f}')

Test macro F1: 0.896970
